# Clone GitHub remote repository to Google Colaboratory (run the following code cell one time only)

In [1]:
import os

def clone(url, dpath=".", branch="master"):
  """
  Clone remote branch(es) from url into dpath
  branch: name of branch to clone ('None' will clone all branches)
  """
  
  url = url.strip("/")
  repo_name = os.path.basename(url)
  os.chdir(dpath)
  
  # Clone specific branch
  if branch:
    !git clone --single-branch --branch "$branch" "$url"
  # Clone all branches
  else:
    !git clone "$url"
    !git branch
    print("Use !git checkout <branchname> to switch to desired branch")
  os.chdir(repo_name)

  print("Current directory: {}". format(os.getcwd()))

Cloning into 'UCSanDiegoX---Machine-Learning-Fundamentals-03-Jan-2019-audit'...
remote: Enumerating objects: 111, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 111 (delta 35), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (111/111), 2.11 MiB | 2.13 MiB/s, done.
Resolving deltas: 100% (35/35), done.
Current directory: /content/UCSanDiegoX---Machine-Learning-Fundamentals-03-Jan-2019-audit


# Push local repository (in Google Colaboratory) to GitHub remote repository

In [0]:
from google.colab import files
import json
from urllib.parse import urlsplit

def push(url, branch="master"):
  """
  Push local branch(es) to repository url
  User provides .json file containing account information
  Text in .json file: {"username": <username>, "email": <email>, "password": <password>, "name": <display name>}
  If name not provided, defaults to username
  branch: name of branch to push ('None' will push all branches)
  """
  
  # Obtain account information
  uploaded = files.upload() # upload .json file
  [fpath] = [*uploaded] # key of dict uploaded is filepath
  with open(fpath) as f:
    account = json.load(f)
  !rm "$fpath" # remove .json file
  account.setdefault("name", account["username"]) # name defaults to username if not provided
  
  # Construct push-authorized url by adding username and password to repository url
  r = urlsplit(url)
  r_auth = r._replace(netloc="{username}:{password}@{hostname}".format(username=account["username"], password=account["password"], hostname=r.hostname))
  url_auth = r_auth.geturl()

  # Set up git
  name, email = account["name"], account["email"]
  !git config --global user.name "$name"
  !git config --global user.email "$email"
  !git remote add origin_auth "$url_auth"
  
  # Push
  if branch:
    input("Press Enter to push {}...".format(branch))
    !git push origin_auth "$branch"
  else:
    input("Press Enter to push all branches...")
    !git push --all origin_auth